In [ ]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm_notebook

warnings.filterwarnings(action='ignore')

DATA_DIR = '/content/drive/MyDrive/투빅스 컨퍼런스/Data/'

In [ ]:
user_review_df = pd.read_csv(DATA_DIR + 'steam_reviews_clean_result.csv')
user_review_df.head()

,content,recommended,user_id,game_id,timestamp,play_time_minute,review_time_minute,game_content_link,review_helpful_count,label_encode_user_id,label_encode_game_id
0,"Very good game, within 6 hours from launch, th...",Recommended,atgbui,768520,2021-06-20,3042.0,3042.0,https://store.steampowered.com/app/768520,5,19302,15392
1,I cannot recommend this game. It's been 8 year...,Not Recommended,atgbui,251570,2021-06-14,40440.0,40440.0,https://store.steampowered.com/app/251570,48,19302,2415
2,Game is currently busted for Co-Op. Despite th...,Not Recommended,atgbui,671510,2021-06-12,1734.0,906.0,https://store.steampowered.com/app/671510,3,19302,13585
3,Unfortunately dead game. https://steamcharts.c...,Not Recommended,atgbui,613590,2021-06-12,48.0,48.0,https://store.steampowered.com/app/613590,0,19302,12513
4,It just felt too generic- why would I play thi...,Not Recommended,atgbui,374280,2021-05-20,540.0,540.0,https://store.steampowered.com/app/374280,1,19302,6269


In [ ]:
user_meta_df = pd.read_csv(DATA_DIR + 'steam_user_meta_data_final.csv')
user_meta_df.head()

,user_id,played_game_total_cnt,played_game_rec_cnt,played_game_not_rec_cnt,played_game_avg_play_time,played_game_rec_avg_play_time,played_game_not_rec_avg_play_time,preference_rec_genre,label_encode_user_id
0,atgbui,50,25,25.0,8192.880000,13536.720000,2849.040000,Multiplayer,19302
1,76561198120102283,19,6,13.0,8528.842105,9270.000000,8186.769231,Action,8135
2,113L1T3G0DM4N3X1T11,10,7,3.0,8719.800000,6888.857143,12992.000000,Action,82
3,_treeheart_,10,6,4.0,9352.200000,15315.000000,408.000000,FPS,18671
4,Jamzuz,8,5,3.0,42954.750000,68553.600000,290.000000,Singleplayer,14352


In [ ]:
game_meta_df = pd.read_csv(DATA_DIR + 'steam_game_meta_data_final.csv')
game_meta_df.head()

,game_id,title,content,Image_url,genres,clean_genres,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags,label_encode_game_id,game_total_cnt,game_rec_cnt,game_not_rec_cnt,recommend_ratio,not_recommend_ratio,game_total_avg_play_time,game_rec_avg_play_time,game_not_rec_avg_play_time
0,10,Counter-Strike,Play the world's number 1 online action game. ...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tAction\t\t\t\t\t...,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",10,Counter-Strike,Valve,Valve,NaN,178066,4597,0,"10,000,000 .. 20,000,000",12516,431,185,764,199.0,999.0,80.0,"English, French, German, Italian, Spanish - Sp...",Action,13111,"{'Action': 5355, 'FPS': 4771, 'Multiplayer': 3...",0,504,475.0,29.0,0.942460,0.057540,42992.559524,45121.048421,8129.379310
1,20,Team Fortress Classic,One of the most popular online action games of...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tAction\t\t\t\t\t...,"['Action', 'FPS', 'Multiplayer', 'Classic', 'H...",20,Team Fortress Classic,Valve,Valve,NaN,4949,838,0,"2,000,000 .. 5,000,000",2353,0,17,0,99.0,499.0,80.0,"English, French, German, Italian, Spanish - Sp...",Action,89,"{'Action': 741, 'FPS': 302, 'Multiplayer': 255...",1,164,130.0,34.0,0.792683,0.207317,11446.682927,14363.538462,294.000000
2,30,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tFPS\t\t\t\t\t\t\...,"['FPS', 'World War II', 'Multiplayer', 'Shoote...",30,Day of Defeat,Valve,Valve,NaN,4723,521,0,"5,000,000 .. 10,000,000",2164,0,39,0,99.0,499.0,80.0,"English, French, German, Italian, Spanish - Spain",Action,122,"{'FPS': 780, 'World War II': 245, 'Multiplayer...",2,107,87.0,20.0,0.813084,0.186916,23939.551402,28533.034483,3957.900000
3,40,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tAction\t\t\t\t\t...,"['Action', 'FPS', 'Classic', 'Multiplayer', 'S...",40,Deathmatch Classic,Valve,Valve,NaN,1733,377,0,"5,000,000 .. 10,000,000",2862,0,11,0,99.0,499.0,80.0,"English, French, German, Italian, Spanish - Sp...",Action,5,"{'Action': 627, 'FPS': 137, 'Classic': 105, 'M...",3,97,78.0,19.0,0.804124,0.195876,14408.845361,16979.000000,3857.684211
4,50,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tFPS\t\t\t\t\t\t\...,"['FPS', 'Action', 'Classic', 'Sci-fi', 'Single...",50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,11285,581,0,"5,000,000 .. 10,000,000",1494,0,208,0,99.0,499.0,80.0,"English, French, German, Korean",Action,137,"{'FPS': 874, 'Action': 316, 'Sci-fi': 242, 'Cl...",4,293,276.0,17.0,0.941980,0.058020,4025.447099,3990.065217,4599.882353


# 데이터셋 나누기

Test data로는 20% 사용할 것이기 때문에

Train data에는 유저의 선호 / 비선호 데이터가 적어도 1개씩은 존재 해야함

그런데 비선호 데이터를 남기지 않은 유저가 존재하기 때문에 훈련시에 네가티브 샘플링(유저가 플레이하지 않은 게임 중에 무작위로 0으로 넣음)을 진행해야함

따라서 선호 리뷰를 2개 이상을 남긴 유저만을 사용




In [ ]:
_user_meta_df = user_meta_df[(user_meta_df['played_game_rec_cnt'] >= 2)].reset_index(drop = True)
_user_meta_df_idx = _user_meta_df['user_id'].tolist()
_user_review_df = user_review_df.set_index('user_id', drop=False).loc[_user_meta_df_idx, :].reset_index(drop = True)

In [ ]:
game_meta_df['game_id'].nunique()

22443

In [ ]:
_user_review_df['game_id'].nunique()

22443

In [ ]:
_user_review_df['user_id'].nunique()

29732

In [ ]:
_user_meta_df['user_id'].nunique()

29732

유저의 변형이 생겼기 때문에 유저 아이디를 새롭게 인코딩 해줌

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
_user_review_df['label_encode_user_id'] = le.fit_transform(_user_review_df['user_id'])

In [ ]:
_user_review_df['label_encode_user_id'].max()

29731

Train과 Test로 분리

In [ ]:
_user_review_df['idx'] = _user_review_df.index
_user_review_df

,content,recommended,user_id,game_id,timestamp,play_time_minute,review_time_minute,game_content_link,review_helpful_count,label_encode_user_id,label_encode_game_id,idx
0,"Very good game, within 6 hours from launch, th...",Recommended,atgbui,768520,2021-06-20,3042.0,3042.0,https://store.steampowered.com/app/768520,5,19157,15392,0
1,I cannot recommend this game. It's been 8 year...,Not Recommended,atgbui,251570,2021-06-14,40440.0,40440.0,https://store.steampowered.com/app/251570,48,19157,2415,1
2,Game is currently busted for Co-Op. Despite th...,Not Recommended,atgbui,671510,2021-06-12,1734.0,906.0,https://store.steampowered.com/app/671510,3,19157,13585,2
3,Unfortunately dead game. https://steamcharts.c...,Not Recommended,atgbui,613590,2021-06-12,48.0,48.0,https://store.steampowered.com/app/613590,0,19157,12513,3
4,It just felt too generic- why would I play thi...,Not Recommended,atgbui,374280,2021-05-20,540.0,540.0,https://store.steampowered.com/app/374280,1,19157,6269,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1076913,good game!!,Recommended,76561198124865235,222880,2015-09-10,924.0,156.0,https://store.steampowered.com/app/222880,0,8238,1767,1076913
1076914,This game head Favorites!,Recommended,76561198124865235,730,2015-09-06,70164.0,40464.0,https://store.steampowered.com/app/730,0,8238,26,1076914
1076915,A really good game! ... At first it was hard t...,Recommended,76561198124865235,274940,2015-09-03,8526.0,696.0,https://store.steampowered.com/app/274940,1,8238,3025,1076915
1076916,good game!!!!!!!!!!!!!!!,Recommended,76561198124865235,238460,2015-07-28,2778.0,2670.0,https://store.steampowered.com/app/238460,0,8238,2124,1076916


In [ ]:
from tqdm import tqdm_notebook

_user_review_df['idx'] = _user_review_df.index
test_idx = []
group_df = _user_review_df.groupby('user_id')

for user_id in tqdm_notebook(group_df.groups):
  all_user_df = group_df.get_group(user_id)
  ret_user_df = all_user_df[all_user_df['recommended'] == 'Recommended'].sort_values('timestamp', ascending = False)
  non_ret_user_df = all_user_df[all_user_df['recommended'] != 'Recommended'].sort_values('timestamp', ascending = False)
  
  # 추천
  ret_split_ratio = int(round(len(ret_user_df) * 0.2))
  if ret_split_ratio <= 1:
    ret_split_ratio = 1

  ret_test_user_df = ret_user_df.iloc[ : ret_split_ratio, :]
  test_idx += ret_test_user_df['idx'].tolist()

  # 비추천
  if len(non_ret_user_df) >= 2:
    non_ret_split_ratio = int(round(len(non_ret_user_df) * 0.2))
    if non_ret_split_ratio <= 1:
      non_ret_split_ratio = 1
    
    non_ret_test_user_df = non_ret_user_df.iloc[ : non_ret_split_ratio, :]
    test_idx += non_ret_test_user_df['idx'].tolist()

In [ ]:
all_idx = _user_review_df['idx'].tolist()
all_idx = set(all_idx)
len(all_idx)

1076918

In [ ]:
test_idx = set(test_idx)
len(test_idx)

218169

In [ ]:
train_idx = all_idx - test_idx
train_idx = list(train_idx)
test_idx = list(test_idx)
len(train_idx)

858749

In [ ]:
train_df = _user_review_df.iloc[train_idx, :].reset_index(drop = True)
test_df = _user_review_df.iloc[test_idx, :].reset_index(drop = True)

In [ ]:
train_df.shape

(858749, 12)

In [ ]:
test_df.shape

(218169, 12)

In [ ]:
train_df['game_id'].nunique()

22049

In [ ]:
train_df['user_id'].nunique()

29732

In [ ]:
test_df['game_id'].nunique()

20025

In [ ]:
test_df['user_id'].nunique()

29732

In [ ]:
train_df.to_csv(DATA_DIR + 'train_df_v2.csv', index = False)
train_df.head()

,content,recommended,user_id,game_id,timestamp,play_time_minute,review_time_minute,game_content_link,review_helpful_count,label_encode_user_id,label_encode_game_id,idx
0,"Game is dead, installed, can play training. Bu...",Not Recommended,atgbui,728540,2021-05-19,18.0,18.0,https://store.steampowered.com/app/728540,0,19157,14662,6
1,Adjust settings- mouse disappears (invisible) ...,Not Recommended,atgbui,999270,2021-05-19,6.0,6.0,https://store.steampowered.com/app/999270,1,19157,18652,7
2,I wish there was a neutral review button.The g...,Not Recommended,atgbui,920470,2021-05-16,216.0,216.0,https://store.steampowered.com/app/920470,0,19157,17716,8
3,"The game brings something new to the genre, an...",Not Recommended,atgbui,1183940,2021-05-09,738.0,738.0,https://store.steampowered.com/app/1183940,0,19157,20641,9
4,Update: 06/11/2021: Game seems to be abandoned...,Not Recommended,atgbui,1254400,2021-05-09,438.0,438.0,https://store.steampowered.com/app/1254400,1,19157,21185,10


In [ ]:
test_df.to_csv(DATA_DIR + 'test_df_v2.csv', index = False)
test_df.head()

,content,recommended,user_id,game_id,timestamp,play_time_minute,review_time_minute,game_content_link,review_helpful_count,label_encode_user_id,label_encode_game_id,idx
0,A great strategy game more focused on grand st...,Recommended,76561198039515085,573410,2020-11-29,12960.0,4602.0,https://store.steampowered.com/app/573410,3,4835,11598,1048576
1,"A very annoying game to play, Needs a lot of q...",Not Recommended,76561198039515085,545330,2020-09-24,120.0,108.0,https://store.steampowered.com/app/545330,1,4835,10900,1048577
2,It's the only co-op game like this you will fi...,Not Recommended,76561198039515085,1132210,2020-05-08,4500.0,2880.0,https://store.steampowered.com/app/1132210,0,4835,20154,1048578
3,"Cute little trading, merchant, workshop chain ...",Recommended,76561198039515085,1040070,2020-04-21,942.0,942.0,https://store.steampowered.com/app/1040070,0,4835,19121,1048579
4,"Very good game, within 6 hours from launch, th...",Recommended,atgbui,768520,2021-06-20,3042.0,3042.0,https://store.steampowered.com/app/768520,5,19157,15392,0


# Train, Val, Test 나누기

- 무작위로 유저를 8 : 2 로 나눈다.

- Train에는 train_df이 사용되기 때문에 모든 유저가 존재함

- test_df에 8에 해당하는 유저를 Val

- test_df에 2에 해당하는 유저를 Test

In [ ]:
import random
from sklearn.model_selection import train_test_split

all_user_id = _user_review_df['user_id'].unique().tolist()
random.shuffle(all_user_id)

val_user_id, test_user_id = train_test_split(all_user_id, train_size=0.8, test_size=0.2)

In [ ]:
len(all_user_id)

29732

In [ ]:
len(val_user_id)

23785

In [ ]:
len(test_user_id)

5947

In [ ]:
_test_df = test_df.set_index('user_id', drop = False)

In [ ]:
val_set = _test_df.loc[val_user_id, :].reset_index(drop = True)
test_set = _test_df.loc[test_user_id, :].reset_index(drop = True)

In [ ]:
test_df['game_id'].nunique()

20025

In [ ]:
test_df['user_id'].nunique()

29732

In [ ]:
val_set['game_id'].nunique()

19223

In [ ]:
val_set['user_id'].nunique()

23785

In [ ]:
test_set['game_id'].nunique()

11514

In [ ]:
test_set['user_id'].nunique()

5947

# 저장

In [ ]:
train_df.to_csv(DATA_DIR + 'steam_reviews_clean_result_train_v2.csv', index = False)
train_df.head()

,content,recommended,user_id,game_id,timestamp,play_time_minute,review_time_minute,game_content_link,review_helpful_count,label_encode_user_id,label_encode_game_id,idx
0,"Game is dead, installed, can play training. Bu...",Not Recommended,atgbui,728540,2021-05-19,18.0,18.0,https://store.steampowered.com/app/728540,0,19157,14662,6
1,Adjust settings- mouse disappears (invisible) ...,Not Recommended,atgbui,999270,2021-05-19,6.0,6.0,https://store.steampowered.com/app/999270,1,19157,18652,7
2,I wish there was a neutral review button.The g...,Not Recommended,atgbui,920470,2021-05-16,216.0,216.0,https://store.steampowered.com/app/920470,0,19157,17716,8
3,"The game brings something new to the genre, an...",Not Recommended,atgbui,1183940,2021-05-09,738.0,738.0,https://store.steampowered.com/app/1183940,0,19157,20641,9
4,Update: 06/11/2021: Game seems to be abandoned...,Not Recommended,atgbui,1254400,2021-05-09,438.0,438.0,https://store.steampowered.com/app/1254400,1,19157,21185,10


In [ ]:
val_set.to_csv(DATA_DIR + 'steam_reviews_clean_result_val_v2.csv', index = False)
val_set.head()

,content,recommended,user_id,game_id,timestamp,play_time_minute,review_time_minute,game_content_link,review_helpful_count,label_encode_user_id,label_encode_game_id,idx
0,Story is a brand of weird that only Kojima can...,Recommended,76561198008676008,1190460,2021-06-04,5322.0,1764.0,https://store.steampowered.com/app/1190460,0,3199,20695,896453
1,"It's more Doom. Rip and tear, etc etc.That be...",Recommended,76561198008676008,782330,2020-05-03,1722.0,222.0,https://store.steampowered.com/app/782330,0,3199,15638,896454
2,"I honestly tried my best to like this game, bu...",Not Recommended,76561198008676008,377160,2017-09-06,1044.0,1044.0,https://store.steampowered.com/app/377160,12,3199,6368,896460
3,Personally I think pc2 is utter trash. Unless ...,Not Recommended,dannyhefc69,378860,2018-02-26,450.0,450.0,https://store.steampowered.com/app/378860,5,20545,6444,903858
4,Don't even think it's scary or nothing. I'm on...,Recommended,dannyhefc69,418370,2018-02-21,684.0,546.0,https://store.steampowered.com/app/418370,1,20545,7688,903859


In [ ]:
test_set.to_csv(DATA_DIR + 'steam_reviews_clean_result_test_v2.csv', index = False)
test_set.head()

,content,recommended,user_id,game_id,timestamp,play_time_minute,review_time_minute,game_content_link,review_helpful_count,label_encode_user_id,label_encode_game_id,idx
0,"Undead Labs, you have a history of disappointi...",Not Recommended,movnviol,495420,2021-04-16,486.0,486.0,https://store.steampowered.com/app/495420,7,24957,9671,3090
1,A good idea and nice aesthetic ruined by non-e...,Not Recommended,movnviol,218640,2021-01-20,306.0,306.0,https://store.steampowered.com/app/218640,18,24957,1696,3091
2,"This is not another shallow early access ""simu...",Recommended,movnviol,682990,2020-04-17,5616.0,390.0,https://store.steampowered.com/app/682990,0,24957,13815,3092
3,I've quite enjoyed the time I've spent on it s...,Recommended,movnviol,916840,2020-01-29,2154.0,156.0,https://store.steampowered.com/app/916840,2,24957,17680,3093
4,It's pretty fun.Lacks the graphics and real-wo...,Recommended,movnviol,485900,2018-12-01,3318.0,2034.0,https://store.steampowered.com/app/485900,5,24957,9410,3096


In [ ]:
_user_meta_df.to_csv(DATA_DIR + 'steam_user_meta_data_final_v2.csv', index = False)
_user_meta_df.head()

,user_id,played_game_total_cnt,played_game_rec_cnt,played_game_not_rec_cnt,played_game_avg_play_time,played_game_rec_avg_play_time,played_game_not_rec_avg_play_time,preference_rec_genre,label_encode_user_id
0,atgbui,50,25,25.0,8192.880000,13536.720000,2849.040000,Multiplayer,19302
1,76561198120102283,19,6,13.0,8528.842105,9270.000000,8186.769231,Action,8135
2,113L1T3G0DM4N3X1T11,10,7,3.0,8719.800000,6888.857143,12992.000000,Action,82
3,_treeheart_,10,6,4.0,9352.200000,15315.000000,408.000000,FPS,18671
4,Jamzuz,8,5,3.0,42954.750000,68553.600000,290.000000,Singleplayer,14352


In [ ]:
_user_review_df.to_csv(DATA_DIR + 'steam_reviews_clean_result_v2.csv', index = False)
_user_review_df.head()

,content,recommended,user_id,game_id,timestamp,play_time_minute,review_time_minute,game_content_link,review_helpful_count,label_encode_user_id,label_encode_game_id,idx
0,"Very good game, within 6 hours from launch, th...",Recommended,atgbui,768520,2021-06-20,3042.0,3042.0,https://store.steampowered.com/app/768520,5,19157,15392,0
1,I cannot recommend this game. It's been 8 year...,Not Recommended,atgbui,251570,2021-06-14,40440.0,40440.0,https://store.steampowered.com/app/251570,48,19157,2415,1
2,Game is currently busted for Co-Op. Despite th...,Not Recommended,atgbui,671510,2021-06-12,1734.0,906.0,https://store.steampowered.com/app/671510,3,19157,13585,2
3,Unfortunately dead game. https://steamcharts.c...,Not Recommended,atgbui,613590,2021-06-12,48.0,48.0,https://store.steampowered.com/app/613590,0,19157,12513,3
4,It just felt too generic- why would I play thi...,Not Recommended,atgbui,374280,2021-05-20,540.0,540.0,https://store.steampowered.com/app/374280,1,19157,6269,4
